# 布洛芬（Ibuprofen）水溶解度预测示例

这个 notebook 演示如何从 SMILES（分子结构）出发，使用传统分子指纹（ECFP/Morgan）与随机森林回归器建立一个溶解度（logS, 单位 mol/L 的对数）预测基线模型，并以布洛芬（Ibuprofen）为例做预测与适用域检测。

要点：
- 使用 RDKit 生成 Morgan/ECFP 指纹
- 使用 scikit-learn 的 RandomForestRegressor 做回归
- 使用公开的 ESOL (Delaney) 溶解度数据集训练模型（notebook 会自动下载或通过 DeepChem 加载）
- 对布洛芬计算预测值并给出与训练集的相似度（Tanimoto）以评估可信度

说明：该 notebook 假定你已安装 RDKit（通常通过 conda 安装最可靠）以及常见 Python 包（pandas, numpy, scikit-learn）。在本仓库也提供了 environment.yml，推荐用 conda 环境来运行。

## 1. 环境 & 安装建议

建议（推荐）使用 conda 创建环境并安装 rdkit：
```bash
conda create -n mol-ml python=3.8 -y
conda activate mol-ml
conda install -c conda-forge rdkit pandas scikit-learn numpy matplotlib joblib -y
pip install deepchem  # 可选：如果想用 deepchem 的数据加载器
```
如果你已在支持 RDKit 的环境（如 conda-forge）中，可以直接运行本 notebook。

In [ ]:
# 导入必要包
import os
import math
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors, DataStructs
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import joblib

print('RDKit version:', Chem.__version__)


## 2. 加载 ESOL（Delaney）数据集
Notebook 尝试以下两种方式获取数据：
1. 如果安装了 DeepChem，则用 deepchem.molnet.load_delaney；
2. 否则尝试从 DeepChem GitHub 仓库的 raw CSV 下载（若下载地址失效，请手动提供 CSV）。


In [ ]:
def load_delaney_df():
    # 先尝试用 deepchem
    try:
        import deepchem as dc
        print('Using DeepChem loader')
        tasks, datasets, transformers = dc.molnet.load_delaney(featurizer=None, split=None)
        # deepchem loader returns a DiskDataset; we'll convert to DataFrame
        # However, simpler: use raw CSV download fallback if this path is awkward
    except Exception as e:
        # 下载 CSV（从 deepchem 仓库的 raw 文件）
        print('DeepChem not available or failed to import:', e)
    
    # 尝试从 GitHub raw 直接下载（备选），若失败请手动提供文件
    urls = [
        'https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv',
        'https://raw.githubusercontent.com/deepchem/deepchem/master/examples/datasets/delaney-processed.csv',
        'https://raw.githubusercontent.com/deepchem/DeepChem/master/datasets/delaney-processed.csv'
    ]
    for url in urls:
        try:
            print('Trying to download from', url)
            df = pd.read_csv(url)
            print('Downloaded dataset from', url)
            return df
        except Exception as e:
            print('Failed to download from', url, '->', e)
    
    raise RuntimeError('无法自动获取 Delaney/ESOL 数据集。请手动下载 delaney-processed.csv 并放在 notebook 同目录下，或安装 deepchem。')

# 调用加载函数
df = load_delaney_df()
df.head()


说明：delaney-processed.csv 通常包含列 ['smiles', 'measured log solubility in mols per litre'] 或者 'logS' 等。我们随后会根据实际列名做适配。

In [ ]:
# 适配和检查列名
def prepare_delaney_dataframe(df):
    df = df.copy()
    # 常见几种列名，选择合适的目标列
    possible_logS = ['measured log solubility in mols per litre', 'logS', 'measured log solubility in mols per litre ']
    target_col = None
    for c in possible_logS:
        if c in df.columns:
            target_col = c
            break
    # 有些版本把列命名为 'Solubility' 或 'measured log S'
    if target_col is None:
        for c in df.columns:
            if 'log' in c.lower() and 'sol' in c.lower():
                target_col = c
                break
    if target_col is None:
        raise RuntimeError('无法识别数据集中的溶解度列，请检查 CSV 列名：\n' + ','.join(df.columns))
    df = df.rename(columns={target_col: 'logS'})
    if 'smiles' not in df.columns:
        raise RuntimeError('CSV 中未找到 smiles 列')
    df = df[['smiles', 'logS']].dropna()
    # 有时 logS 列是字符串，转换为 float
    df['logS'] = df['logS'].astype(float)
    return df

df = prepare_delaney_dataframe(df)
print('Number of molecules in ESOL (Delaney):', len(df))
df.head()


## 3. 将 SMILES 转为 Morgan/ECFP 指纹（位向量）

In [ ]:
def smiles_to_ecfp_bitvec(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    arr = np.zeros((nBits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# 生成特征矩阵
nBits = 2048
X_list = []
y_list = []
valid_smiles = []
for smi, y in zip(df['smiles'], df['logS']):
    arr = smiles_to_ecfp_bitvec(smi, radius=2, nBits=nBits)
    if arr is None:
        continue
    X_list.append(arr)
    y_list.append(y)
    valid_smiles.append(smi)

X = np.vstack(X_list)
y = np.array(y_list)
print('Feature matrix shape:', X.shape)


## 4. 划分数据集并训练随机森林回归模型

In [ ]:
X_train, X_test, y_train, y_test, smiles_train, smiles_test = train_test_split(
    X, y, valid_smiles, test_size=0.2, random_state=42
)
print('Train size:', X_train.shape[0], 'Test size:', X_test.shape[0])

model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# 评估
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Test RMSE = {rmse:.3f}, MAE = {mae:.3f}, R2 = {r2:.3f}')

# 简单的预测可视化：真实 vs 预测
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Measured logS (mol/L)')
plt.ylabel('Predicted logS (mol/L)')
plt.title('RandomForest (ECFP) Predictions')
plt.grid(True)
plt.show()


## 5. 用模型预测布洛芬（Ibuprofen）的溶解度，并给出适用域指标（Tanimoto 相似度）

布洛芬 SMILES（常用表示）: CC(C)CC1=CC=C(C=C1)C(C)C(=O)O

In [ ]:
ibuprofen_smiles = 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O'
ibuprofen_fp = smiles_to_ecfp_bitvec(ibuprofen_smiles, radius=2, nBits=nBits)
pred_logS_ibuprofen = model.predict(ibuprofen_fp.reshape(1, -1))[0]
print(f'Predicted logS (mol/L) for ibuprofen: {pred_logS_ibuprofen:.3f} (log10 mol/L)')

# 适用域检测：计算与训练集中指纹的最大 Tanimoto 相似度
def bitvec_to_rdkit_fp(bitarr):
    # bitarr: numpy array of 0/1
    arr = np.asarray(bitarr, dtype=np.uint8)
    bv = DataStructs.ExplicitBitVect(len(arr))
    for i, v in enumerate(arr):
        if int(v):
            bv.SetBit(i)
    return bv

train_bitvecs = [bitvec_to_rdkit_fp(x) for x in X_train]
ib_fp_rd = bitvec_to_rdkit_fp(ibuprofen_fp)
sims = [DataStructs.TanimotoSimilarity(ib_fp_rd, t) for t in train_bitvecs]
max_sim = max(sims)
mean_sim = float(np.mean(sims))
print(f'Max Tanimoto similarity to training set: {max_sim:.3f}, mean similarity: {mean_sim:.3f}')

# 将预测 logS 转换为 mol/L 与 mg/L（需要分子量）
mol = Chem.MolFromSmiles(ibuprofen_smiles)
mw = Descriptors.MolWt(mol)
s_mol_per_l = 10 ** pred_logS_ibuprofen
s_mg_per_l = s_mol_per_l * mw * 1000.0
print(f'Predicted solubility: {s_mol_per_l:.3e} mol/L = {s_mg_per_l:.3f} mg/L (MW={mw:.3f} g/mol)')


说明：如果 ibuprofen 与训练集的最大 Tanimoto 相似度较高（例如 > 0.6），则基线预测更可信；若很低，则需要谨慎对待预测值。

你可以通过 ensembling（多个模型）或做不确定性估计（模型集成、bootstrap、MC-dropout）来获取置信区间。

## 6. 保存模型以便后续快速预测
我们把训练好的随机森林模型和训练时使用的参数（比如 nBits）一起保存。

In [ ]:
model_path = 'rf_ecfp_model.joblib'
joblib.dump({'model': model, 'nBits': nBits, 'radius': 2}, model_path)
print('Saved model to', model_path)


## 7. 总结与后续建议

- 本 notebook 给出了一个可复现的 baseline：ECFP（Morgan 指纹）+ RandomForest，用于预测溶解度（logS）。
- 若你想更进一步：
  - 用交叉验证调参（GridSearchCV）优化随机森林参数；
  - 尝试更强的学习型指纹（如 chemprop 的 D-MPNN、DeepChem 的 GraphConv/MPNN），尤其是当数据量较大时；
  - 做不确定性估计与适用域评估；
  - 若你的目标单位是 mg/L，请使用分子量做单位换算（notebook 已示范）。

如果你希望，我可以：
- 把这个 notebook 放到 GitHub 仓库并添加 Binder/Colab 运行说明；
- 把同样流程改成 DeepChem / GraphConv 的对比 notebook；
- 或者把模型部署成一个小的脚本/Flask API，便于对任意 SMILES 做预测。

你希望我接下来做哪一步？